In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName('lab04').getOrCreate()

22/01/21 17:04:47 WARN Utils: Your hostname, sheepb-HP-Pavilion-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlo1)
22/01/21 17:04:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 17:04:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/21 17:04:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## a. Đọc và xử lí dữ liệu

In [3]:
data_path = './Lab04-Data/'
order_csv = spark.read.csv(data_path + 'orders.csv', inferSchema=True, header=True, sep=',')
product_csv = spark.read.csv(data_path + 'products.csv', inferSchema=True, header=True, sep=',')

In [4]:
order_csv.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
|       1|     13176|                6|        0|
|       1|     47209|                7|        0|
|       1|     22035|                8|        1|
|      36|     39612|                1|        0|
|      36|     19660|                2|        1|
|      36|     49235|                3|        0|
|      36|     43086|                4|        1|
|      36|     46620|                5|        1|
|      36|     34497|                6|        1|
|      36|     48679|                7|        1|
|      36|     46979|                8|        1|
|      38|     11913|                1|        0|


In [5]:
product_csv.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

- merge 2 bảng lại như yêu cầu đề bài

In [6]:
merge_df =  order_csv.join(product_csv, 
                             order_csv.product_id == product_csv.product_id)\
                        .groupby('order_id')\
                        .agg(F.collect_list('product_name'))
merge_df.show()

+--------+--------------------------+
|order_id|collect_list(product_name)|
+--------+--------------------------+
|       1|      [Bulgarian Yogurt...|
|     218|      [Natural Artisan ...|
|     473|      [Organic Whole Mi...|
|     631|      [Organic Strawber...|
|     762|      [Organic Strawber...|
|     774|      [Ice Cream Variet...|
|     844|      [Green Beans, Org...|
|     988|      [Natural Vanilla ...|
|    1077|      [Bag of Organic B...|
|    1139|      [Banana, Organic ...|
|    1143|      [Natural Premium ...|
|    1145|      [Banana, Original...|
|    1280|      [Lactose Free Hal...|
|    1342|      [Bag of Organic B...|
|    1350|      [Ground Cinnamon,...|
|    1468|      [Banana, Organic ...|
|    1591|      [Cracked Wheat, O...|
|    1682|      [Bathroom Tissue,...|
|    1721|      [Organic Reduced ...|
|    1890|      [Vanilla Almond B...|
+--------+--------------------------+
only showing top 20 rows



- thay đổi lại tên cột theo yêu cầu

In [7]:
df = merge_df.withColumnRenamed("order_id", "id")\
       .withColumnRenamed("collect_list(product_name)", "products")
df.show()

+----+--------------------+
|  id|            products|
+----+--------------------+
|   1|[Bulgarian Yogurt...|
| 218|[Natural Artisan ...|
| 473|[Organic Whole Mi...|
| 631|[Organic Strawber...|
| 762|[Organic Strawber...|
| 774|[Ice Cream Variet...|
| 844|[Green Beans, Org...|
| 988|[Natural Vanilla ...|
|1077|[Bag of Organic B...|
|1139|[Banana, Organic ...|
|1143|[Natural Premium ...|
|1145|[Banana, Original...|
|1280|[Lactose Free Hal...|
|1342|[Bag of Organic B...|
|1350|[Ground Cinnamon,...|
|1468|[Banana, Organic ...|
|1591|[Cracked Wheat, O...|
|1682|[Bathroom Tissue,...|
|1721|[Organic Reduced ...|
|1890|[Vanilla Almond B...|
+----+--------------------+
only showing top 20 rows



## b. Khai thác tập phổ biến

In [8]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- products: array (nullable = false)
 |    |-- element: string (containsNull = false)



- Trước tiên ta thử với minSupport=0.8, minConfidence=0.8

In [9]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="products", minSupport=0.8, minConfidence=0.8)
model = fpGrowth.fit(df)
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

/home/sheepb/code/jupyter_dir/jupyter_env/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+----+
|items|freq|
+-----+----+
+-----+----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[Bulgarian Yogurt...|        []|
| 218|[Natural Artisan ...|        []|
| 473|[Organic Whole Mi...|        []|
| 631|[Organic Strawber...|        []|
| 762|[Organic Strawber...|        []|
| 774|[Ice Cream Variet...|        []|
| 844|[Green Beans, Org...|        []|
| 988|[Natural Vanilla ...|        []|
|1077|[Bag of Organic B...|        []|
|1139|[Banana, Organic ...|        []|
|1143|[Natural Premium ...|        []|
|1145|[Banana, Original...|        []|
|1280|[Lactose Free Hal...|        []|
|1342|[Bag of Organic B...|        []|
|1350|[Ground Cinnamon,...|        []|
|1468|[Banana, Organic ...|        []|
|1591|[Cracked Wheat, O...|        []|
|1682|[Bathroom Tissue,...|        []|
|1721|[Organic Reduced ...|        []|
|1890|[Vanilla Almond B...|        []|
+----+--------------------+----------+
only showing top 20 rows



- Thử với các giá trị min_support, min_confidence nhỏ hơn

In [10]:
test_parameter = [(0.6, 0.6), (0.6, 0.4), (0.3, 0.5), (0.4, 0.1), (0.1, 0.1), (0.01, 0.01)]
for min_support, min_confidence in test_parameter:
    print(f"------------- support={min_support}, confidence={min_confidence}--------------")
    fpGrowth = FPGrowth(itemsCol="products", minSupport=min_support, minConfidence=min_confidence)
    model = fpGrowth.fit(df)
    # Display frequent itemsets.
    model.freqItemsets.show()
    
    # Display generated association rules.
    model.associationRules.show()
    
    # transform examines the input items against all the association rules and summarize the
    # consequents as prediction
    model.transform(df).show()

------------- support=0.6, confidence=0.6--------------


+-----+----+
|items|freq|
+-----+----+
+-----+----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[Bulgarian Yogurt...|        []|
| 218|[Natural Artisan ...|        []|
| 473|[Organic Whole Mi...|        []|
| 631|[Organic Strawber...|        []|
| 762|[Organic Strawber...|        []|
| 774|[Ice Cream Variet...|        []|
| 844|[Green Beans, Org...|        []|
| 988|[Natural Vanilla ...|        []|
|1077|[Bag of Organic B...|        []|
|1139|[Banana, Organic ...|        []|
|1143|[Natural Premium ...|        []|
|1145|[Banana, Original...|        []|
|1280|[Lactose Free Hal...|        []|
|1342|[Bag of Organic B...|        []|
|1350|[Ground Cinnamon,...|        []|
|1468|[Banana, Organic ...|        []|
|1591|[Cracked Wheat, O...|        []|
|1682|[Bathroom Tissue,...|        []|
|1721|[Organic Reduced ...|        []|
|1890|[Vanilla Almond B...|        []|
+----+--------------------+----------+
only showing top 20 rows

------------- support=0.6, confidence=

+-----+----+
|items|freq|
+-----+----+
+-----+----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[Bulgarian Yogurt...|        []|
| 218|[Natural Artisan ...|        []|
| 473|[Organic Whole Mi...|        []|
| 631|[Organic Strawber...|        []|
| 762|[Organic Strawber...|        []|
| 774|[Ice Cream Variet...|        []|
| 844|[Green Beans, Org...|        []|
| 988|[Natural Vanilla ...|        []|
|1077|[Bag of Organic B...|        []|
|1139|[Banana, Organic ...|        []|
|1143|[Natural Premium ...|        []|
|1145|[Banana, Original...|        []|
|1280|[Lactose Free Hal...|        []|
|1342|[Bag of Organic B...|        []|
|1350|[Ground Cinnamon,...|        []|
|1468|[Banana, Organic ...|        []|
|1591|[Cracked Wheat, O...|        []|
|1682|[Bathroom Tissue,...|        []|
|1721|[Organic Reduced ...|        []|
|1890|[Vanilla Almond B...|        []|
+----+--------------------+----------+
only showing top 20 rows

------------- support=0.3, confidence=

...

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
+--------------------+-----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[Bulgarian Yogurt...|        []|
| 218|[Natural Artisan ...|        []|
| 473|[Organic Whole Mi...|        []|
| 631|[Organic Strawber...|        []|
| 762|[Organic Strawber...|        []|
| 774|[Ice Cream Variet...|        []|
| 844|[Green Beans, Org...|        []|
| 988|[Natural Vanilla ...|        []|
|1077|[Bag of Organic B...|        []|
|1139|[Banana, Organic ...|        []|
|1143|[Natural Premium ...|        []|
|1145|[Banana, Original...|        []|
|1280|[Lactose Free Hal...|        []|
|1342|[Bag of Organic B...|        []|
|1350|[Ground Cinnamon,...|        []|
|1468|[Banana, Organic ...|        []|
|1591|[Cracked Wheat, O...|        []|
|1682|[Bathroom Tissue,...|        []|
|1721|[Organic Reduced ...|        []|
|1890|[Vanilla Almond B...|        []|
+----+--------------------+----------+
only showing top 20 rows

------------- support=0.01, confidence

+--------------------+----+
|               items|freq|
+--------------------+----+
|      [Green Onions]|1445|
|  [Jalapeno Peppers]|1899|
|[Organic Large Ex...|2891|
|             [Limes]|6033|
|[Limes, Large Lemon]|1595|
|     [Limes, Banana]|1331|
|[Organic Peeled W...|2460|
|      [Hass Avocado]|1633|
|      [Spring Water]|2225|
|     [Yellow Onions]|3762|
|[Organic Baby Bro...|1351|
|[Lime Sparkling W...|1966|
|[Organic Garnet S...|2568|
|       [Large Lemon]|8135|
|[Large Lemon, Ban...|2158|
|[Organic Lacinato...|1761|
|[Seedless Red Gra...|4059|
| [Pure Irish Butter]|1476|
|[100% Whole Wheat...|2298|
|[Small Hass Avocado]|3103|
+--------------------+----+
only showing top 20 rows



+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|       [Large Lemon]|             [Limes]|0.19606637984019668| 4.264159395400691|0.012156178310939037|
|       [Large Lemon]|            [Banana]|0.26527350952673634|1.8587136554252668|0.016447042504706234|
|       [Large Lemon]|   [Organic Avocado]|0.16582667486170866| 2.936692155747055|0.010281306922543422|
|[Organic Strawber...|[Organic Raspberr...|0.15329539195887645|3.6267102566772844|0.012727785441547455|
|[Organic Strawber...|[Bag of Organic B...|0.28217367358178813|2.3917135359814496|0.023428270926537053|
|[Organic Strawber...|            [Banana]|0.19955939049017807|1.3982691480735754|0.016568985359236028|
|[Organic Strawber...|[Organic Hass Avo...| 0.1412704240866532|2

+----+--------------------+--------------------+
|  id|            products|          prediction|
+----+--------------------+--------------------+
|   1|[Bulgarian Yogurt...|[Organic Raspberr...|
| 218|[Natural Artisan ...|                  []|
| 473|[Organic Whole Mi...|[Limes, Large Lem...|
| 631|[Organic Strawber...|[Organic Raspberr...|
| 762|[Organic Strawber...|[Organic Raspberr...|
| 774|[Ice Cream Variet...|                  []|
| 844|[Green Beans, Org...|                  []|
| 988|[Natural Vanilla ...|                  []|
|1077|[Bag of Organic B...|[Organic Raspberr...|
|1139|[Banana, Organic ...|[Organic Raspberr...|
|1143|[Natural Premium ...|[Limes, Banana, O...|
|1145|[Banana, Original...|[Limes, Large Lem...|
|1280|[Lactose Free Hal...|                  []|
|1342|[Bag of Organic B...|[Organic Raspberr...|
|1350|[Ground Cinnamon,...|                  []|
|1468|[Banana, Organic ...|[Limes, Large Lem...|
|1591|[Cracked Wheat, O...|[Limes, Large Lem...|
|1682|[Bathroom Tiss

- Kết qủa không như mong đợi, không tìm thấy tập phổ biến mặc dù min_support và min_confidence vô cùng thấp (0.1, 0.1)
- Thử với min_support=0.01 và min_confidence=0.01 thì ta tìm được các tập phổ biến

- Thử so sánh số lượng giá trị distinct của product name giữa origin và trường hợp không phân việt hoa thường

In [15]:
product_csv.select('product_name').distinct().count()

49688

In [16]:
product_csv.select(F.lower(product_csv.product_name)).distinct().count()

49588

Có sự chênh lệch ở đây

In [17]:
product_csv = product_csv.withColumn('product_name', F.lower(product_csv.product_name))
product_csv.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|chocolate sandwic...|      61|           19|
|         2|    all-seasons salt|     104|           13|
|         3|robust golden uns...|      94|            7|
|         4|smart ones classi...|      38|            1|
|         5|green chile anyti...|       5|           13|
|         6|        dry nose oil|      11|           11|
|         7|pure coconut wate...|      98|            7|
|         8|cut russet potato...|     116|            1|
|         9|light strawberry ...|     120|           16|
|        10|sparkling orange ...|     115|            7|
|        11|   peach mango juice|      31|            7|
|        12|chocolate fudge l...|     119|            1|
|        13|   saline nasal mist|      11|           11|
|        14|fresh scent dishw...|      74|           17|
|        15|overnight diapers..

In [21]:
merge_df =  order_csv.join(product_csv, 
                             order_csv.product_id == product_csv.product_id)\
                        .groupby('order_id')\
                        .agg(F.collect_set('product_name'))
merge_df.show()

+--------+-------------------------+
|order_id|collect_set(product_name)|
+--------+-------------------------+
|       1|     [organic 4% milk ...|
|     218|     [natural artisan ...|
|     473|     [spinaci e ricott...|
|     631|     [organic red radi...|
|     762|     [celery hearts, o...|
|     774|     [nacho cheese sau...|
|     844|     [baby spinach, or...|
|     988|     [whipped light cr...|
|    1077|     [celery sticks, o...|
|    1139|     [flaky biscuits, ...|
|    1143|     [large lemon, org...|
|    1145|     [harvest best in ...|
|    1280|     [vanilla soy milk...|
|    1342|     [seedless cucumbe...|
|    1350|     [plus lotion faci...|
|    1468|     [organic red radi...|
|    1591|     [honey graham sna...|
|    1682|     [organic yellow o...|
|    1721|     [organic thompson...|
|    1890|     [michigan organic...|
+--------+-------------------------+
only showing top 20 rows



In [24]:
new_df = merge_df.withColumnRenamed("order_id", "id")\
       .withColumnRenamed("collect_set(product_name)", "products")
new_df.show()

+----+--------------------+
|  id|            products|
+----+--------------------+
|   1|[organic 4% milk ...|
| 218|[natural artisan ...|
| 473|[spinaci e ricott...|
| 631|[organic red radi...|
| 762|[celery hearts, o...|
| 774|[nacho cheese sau...|
| 844|[baby spinach, or...|
| 988|[whipped light cr...|
|1077|[celery sticks, o...|
|1139|[flaky biscuits, ...|
|1143|[large lemon, org...|
|1145|[harvest best in ...|
|1280|[vanilla soy milk...|
|1342|[seedless cucumbe...|
|1350|[plus lotion faci...|
|1468|[organic red radi...|
|1591|[honey graham sna...|
|1682|[organic yellow o...|
|1721|[organic thompson...|
|1890|[michigan organic...|
+----+--------------------+
only showing top 20 rows



In [26]:
test_parameter = [(0.1, 0.1), (0.05, 0.05), (0.01, 0.01)]
for min_support, min_confidence in test_parameter:
    print(f"------------- support={min_support}, confidence={min_confidence}--------------")
    fpGrowth = FPGrowth(itemsCol="products", minSupport=min_support, minConfidence=min_confidence)
    model = fpGrowth.fit(new_df)
    # Display frequent itemsets.
    model.freqItemsets.show()
    
    # Display generated association rules.
    model.associationRules.show()
    
    # transform examines the input items against all the association rules and summarize the
    # consequents as prediction
    model.transform(new_df).show()

------------- support=0.1, confidence=0.1--------------


+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [banana]|18726|
|[bag of organic b...|15480|
+--------------------+-----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[organic 4% milk ...|        []|
| 218|[natural artisan ...|        []|
| 473|[spinaci e ricott...|        []|
| 631|[organic red radi...|        []|
| 762|[celery hearts, o...|        []|
| 774|[nacho cheese sau...|        []|
| 844|[baby spinach, or...|        []|
| 988|[whipped light cr...|        []|
|1077|[celery sticks, o...|        []|
|1139|[flaky biscuits, ...|        []|
|1143|[large lemon, org...|        []|
|1145|[harvest best in ...|        []|
|1280|[vanilla soy milk...|        []|
|1342|[seedless cucumbe...|        []|
|1350|[plus lotion faci...|        []|
|1468|[organic red radi...|        []|
|1591|[honey graham sna...|        []|
|1682|[organic yellow o...|        []|
|1721|[organic thompson...|        []|
|1890|[michigan organic...|        []|
+----+--------------------+----------+
only showing top 20 rows

------------- support=0.05, confidence

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|       [large lemon]| 8135|
|            [banana]|18726|
|   [organic avocado]| 7409|
|[bag of organic b...|15480|
|[organic strawber...|10894|
|[organic hass avo...| 7293|
|[organic baby spi...| 9784|
+--------------------+-----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



+----+--------------------+----------+
|  id|            products|prediction|
+----+--------------------+----------+
|   1|[organic 4% milk ...|        []|
| 218|[natural artisan ...|        []|
| 473|[spinaci e ricott...|        []|
| 631|[organic red radi...|        []|
| 762|[celery hearts, o...|        []|
| 774|[nacho cheese sau...|        []|
| 844|[baby spinach, or...|        []|
| 988|[whipped light cr...|        []|
|1077|[celery sticks, o...|        []|
|1139|[flaky biscuits, ...|        []|
|1143|[large lemon, org...|        []|
|1145|[harvest best in ...|        []|
|1280|[vanilla soy milk...|        []|
|1342|[seedless cucumbe...|        []|
|1350|[plus lotion faci...|        []|
|1468|[organic red radi...|        []|
|1591|[honey graham sna...|        []|
|1682|[organic yellow o...|        []|
|1721|[organic thompson...|        []|
|1890|[michigan organic...|        []|
+----+--------------------+----------+
only showing top 20 rows

------------- support=0.01, confidence

+--------------------+----+
|               items|freq|
+--------------------+----+
|      [green onions]|1445|
|  [jalapeno peppers]|1899|
|[organic large ex...|2891|
|             [limes]|6033|
|[limes, large lemon]|1595|
|     [limes, banana]|1331|
|[organic peeled w...|2460|
|      [hass avocado]|1633|
|      [spring water]|2225|
|     [yellow onions]|3762|
|[organic baby bro...|1351|
|[lime sparkling w...|1966|
|[organic garnet s...|2568|
|       [large lemon]|8135|
|[large lemon, ban...|2158|
|[organic lacinato...|1761|
|[seedless red gra...|4059|
| [pure irish butter]|1476|
|[100% whole wheat...|2298|
|[small hass avocado]|3103|
+--------------------+----+
only showing top 20 rows



+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|      [strawberries]|            [banana]|0.29996920234062213| 2.101818811807684| 0.01484654253900266|
|            [banana]|             [limes]| 0.0710776460536153|1.5458357137491812|0.010144121211197402|
|            [banana]|       [large lemon]|0.11524084161059489|1.8587136554252666|0.016447042504706234|
|            [banana]|   [organic avocado]|0.11833813948520773| 2.095698332260038|0.016889085352376744|
|            [banana]|[organic strawber...|0.11609526861048809|1.3982691480735754|0.016568985359236028|
|            [banana]|      [strawberries]|0.10402648723699669| 2.101818811807684| 0.01484654253900266|
|            [banana]|[organic baby spi...|0.10680337498664957|1

+----+--------------------+--------------------+
|  id|            products|          prediction|
+----+--------------------+--------------------+
|   1|[organic 4% milk ...|[organic strawber...|
| 218|[natural artisan ...|                  []|
| 473|[spinaci e ricott...|[limes, large lem...|
| 631|[organic red radi...|[organic raspberr...|
| 762|[celery hearts, o...|[organic raspberr...|
| 774|[nacho cheese sau...|                  []|
| 844|[baby spinach, or...|                  []|
| 988|[whipped light cr...|                  []|
|1077|[celery sticks, o...|[organic raspberr...|
|1139|[flaky biscuits, ...|[limes, large lem...|
|1143|[large lemon, org...|[organic strawber...|
|1145|[harvest best in ...|[limes, large lem...|
|1280|[vanilla soy milk...|                  []|
|1342|[seedless cucumbe...|[organic raspberr...|
|1350|[plus lotion faci...|                  []|
|1468|[organic red radi...|[limes, large lem...|
|1591|[honey graham sna...|[limes, large lem...|
|1682|[organic yello